# Rules testing
---
Using different rules formats to try and even out the player scores 

In [1]:
import pandas as pd 
import numpy as np 

In [31]:
df_batting = pd.read_csv('./baseballdatabank-master/core/Batting.csv')
df_pitching = pd.read_csv('./baseballdatabank-master/core/Pitching.csv')
df_people = pd.read_csv('./baseballdatabank-master/core/People.csv')

In [3]:
def open_rules(loc):
    with open(loc) as f:
        return eval(f.read())

In [6]:
rules_batting = open_rules('./rules/batting1.txt')
rules_pitching = open_rules('./rules/pitching1.txt')

## Batters investigation

In [55]:
# Join actual batters name
df_people_reduced = df_people[['playerID', 'nameFirst', 'nameLast']].copy()
df_batting_ppl = df_batting.merge(df_people_reduced, how='inner', on='playerID')

In [56]:
identifiers_batting = ['playerID', 'nameFirst', 'nameLast', 'yearID']
columns_batting = identifiers_batting + ['G', 'AB'] + list(rules_batting.keys())
min_year = 2015

In [57]:
# Create a column for singles
df_batting_ppl['1B'] = df_batting_ppl['H'] - df_batting_ppl['2B'] - df_batting_ppl['3B'] - df_batting_ppl['HR']

In [79]:
agg_batting = df_batting_ppl[df_batting_ppl['yearID'] >= min_year][columns_batting].groupby(identifiers_batting).sum().reset_index()

In [80]:
agg_batting[agg_batting['AB'] > 50]

,playerID,nameFirst,nameLast,yearID,G,AB,BB,HBP,SO,1B,2B,3B,HR,RBI,R
8,abreujo02,Jose,Abreu,2015,154,613,39,15.0,140.0,111,34,3,30,101.0,88
9,abreujo02,Jose,Abreu,2016,159,624,47,15.0,125.0,125,32,1,25,100.0,67
10,abreujo02,Jose,Abreu,2017,156,621,35,15.0,119.0,107,43,6,33,102.0,95
11,abreujo02,Jose,Abreu,2018,128,499,37,11.0,109.0,73,36,1,22,78.0,68
12,abreujo02,Jose,Abreu,2019,159,634,36,13.0,152.0,108,38,1,33,123.0,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8129,zuninmi01,Mike,Zunino,2016,55,164,21,6.0,65.0,15,7,0,12,31.0,16
8130,zuninmi01,Mike,Zunino,2017,124,387,39,8.0,160.0,47,25,0,25,64.0,52
8131,zuninmi01,Mike,Zunino,2018,113,373,24,6.0,150.0,37,18,0,20,44.0,37
8132,zuninmi01,Mike,Zunino,2019,90,266,20,3.0,98.0,24,10,1,9,32.0,30


In [87]:
agg_batting['points_total'] = 0

for stat in rules_batting:
    agg_batting[stat + '_points'] = agg_batting[stat] * rules_batting[stat]
    agg_batting['points_total'] += agg_batting[stat + '_points']

agg_batting['ppg'] = agg_batting['points_total'] / agg_batting['G']

In [88]:
agg_batting

,playerID,nameFirst,nameLast,yearID,G,AB,BB,HBP,SO,1B,...,BB_points,HBP_points,SO_points,1B_points,2B_points,3B_points,HR_points,RBI_points,R_points,ppg
0,aardsda01,David,Aardsma,2015,33,1,0,0.0,1.0,0,...,0.0,0.0,-0.5,0,0,0,0,0.0,0,-0.015152
1,abadfe01,Fernando,Abad,2015,62,0,0,0.0,0.0,0,...,0.0,0.0,-0.0,0,0,0,0,0.0,0,0.000000
2,abadfe01,Fernando,Abad,2016,57,1,0,0.0,1.0,0,...,0.0,0.0,-0.5,0,0,0,0,0.0,0,-0.008772
3,abadfe01,Fernando,Abad,2017,48,0,0,0.0,0.0,0,...,0.0,0.0,-0.0,0,0,0,0,0.0,0,0.000000
4,abadfe01,Fernando,Abad,2019,21,0,0,0.0,0.0,0,...,0.0,0.0,-0.0,0,0,0,0,0.0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8132,zuninmi01,Mike,Zunino,2019,90,266,20,3.0,98.0,24,...,10.0,4.5,-49.0,24,20,3,90,160.0,120,4.250000
8133,zuninmi01,Mike,Zunino,2020,28,75,6,3.0,37.0,3,...,3.0,4.5,-18.5,3,8,0,40,50.0,32,4.357143
8134,zychto01,Tony,Zych,2015,13,0,0,0.0,0.0,0,...,0.0,0.0,-0.0,0,0,0,0,0.0,0,0.000000
8135,zychto01,Tony,Zych,2016,12,0,0,0.0,0.0,0,...,0.0,0.0,-0.0,0,0,0,0,0.0,0,0.000000


In [90]:
agg_batting[(agg_batting['AB'] > 50) & (agg_batting['yearID'] == 2019)].sort_values(by='ppg', ascending=False).iloc[1:50]

,playerID,nameFirst,nameLast,yearID,G,AB,BB,HBP,SO,1B,...,BB_points,HBP_points,SO_points,1B_points,2B_points,3B_points,HR_points,RBI_points,R_points,ppg
1623,cruzne02,Nelson,Cruz,2019,120,454,56,7.0,131.0,74,...,28.0,10.5,-65.5,74,52,0,410,540.0,324,11.441667
8060,yelicch01,Christian,Yelich,2019,130,489,80,8.0,118.0,85,...,40.0,12.0,-59.0,85,58,9,440,485.0,400,11.307692
6032,rendoan01,Anthony,Rendon,2019,146,545,80,12.0,86.0,93,...,40.0,18.0,-43.0,93,88,9,340,630.0,468,11.253425
6989,springe01,George,Springer,2019,122,479,67,6.0,113.0,78,...,33.5,9.0,-56.5,78,40,9,390,480.0,384,11.204918
210,alvaryo01,Yordan,Alvarez,2019,87,313,52,2.0,94.0,45,...,26.0,3.0,-47.0,45,52,0,270,390.0,232,11.160920
594,bellico01,Cody,Bellinger,2019,156,558,95,3.0,108.0,86,...,47.5,4.5,-54.0,86,68,9,470,575.0,484,10.833333
2119,encared01,Edwin,Encarnacion,2019,109,418,58,7.0,103.0,50,...,29.0,10.5,-51.5,50,36,0,340,430.0,324,10.715596
880,bregmal01,Alex,Bregman,2019,156,554,119,9.0,83.0,84,...,59.5,13.5,-41.5,84,74,6,410,560.0,488,10.599359
2614,garvemi01,Mitch,Garver,2019,93,311,41,5.0,87.0,37,...,20.5,7.5,-43.5,37,32,3,310,335.0,280,10.553763
1858,deverra01,Rafael,Devers,2019,156,647,48,4.0,119.0,111,...,24.0,6.0,-59.5,111,108,12,320,575.0,516,10.336538


In [32]:
df_people.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,2021.0,1.0,22.0,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01
